<a href="https://colab.research.google.com/github/ZuowenWang0000/Computational-Intelligence-Lab/blob/word-graph/make_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_neg_path = "/content/drive/My Drive/cil-data/train_neg.txt"
train_neg_full_path = "/content/drive/My Drive/cil-data/train_neg_full.txt"
train_pos_path = "/content/drive/My Drive/cil-data/train_pos.txt"
train_pos_full_path = "/content/drive/My Drive/cil-data/train_pos_full.txt"

with open(train_neg_full_path, 'r') as f_pos:
    neg_text = f_pos.read().splitlines()
with open(train_pos_full_path, 'r') as f_neg:
    pos_text = f_neg.read().splitlines()

In [6]:
!pip install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip install fastai

In [0]:
import fastai
from fastai import *
from fastai.text import * 

#tokenizer = Tokenizer()
#tokens_neg = tokenizer.process_all(neg_text)
#tokens_pos = tokenizer.process_all(pos_text)



In [0]:
def clean_sentences(sentence_list):
  fin_list = []
  for s in sentence_list:
    sp = spec_add_spaces(rm_useless_spaces(replace_wrep(replace_rep(fix_html(s)))))
    fin_list.append(sp)
  fin_list = deal_caps(replace_all_caps(fin_list))
  return fin_list

In [0]:
import spacy
def make_graphs(sentence_list):
 
  #returns a list of adjacency matrices

  nlp = spacy.load("en")
  graphs = []
  token_list = []
  max_len = 0
  tot_iter = len(sentence_list)
  for iter,raw_sentence in enumerate(sentence_list):
    if iter % 100 == 0:
      print("ITER: {} of {}".format(iter,tot_iter))
    parsed = nlp(raw_sentence)
    tok_tmp = []
    for token in parsed:
      tok_tmp.append(token)
    token_list.append(tok_tmp)
    l = len(parsed)
    if l > max_len:
      max_len = l
    g = np.zeros((l,l))
    for thing in parsed.sents:
      for tok in thing:
        index = tok.i - thing.start
        s = [x.i for x in tok.children]
        for v in s:
          g[index,v] = 1
          g[v,index] = 1

    graphs.append(g)
  return graphs, token_list, max_len

In [33]:
pos_graphs, pos_tok, pos_max_len = make_graphs(clean_sentences(pos_text[:200]))
neg_graphs, neg_tok, neg_max_len = make_graphs(clean_sentences(neg_text[:200]))

ITER: 0 of 200
ITER: 100 of 200
ITER: 0 of 200
ITER: 100 of 200


In [0]:
def make_graphs_same_length(graph_list, max_len):
  ret_list = []
  for g in graph_list:
    new_g = np.zeros((max_len,max_len))
    d,_ = g.shape
    new_g[:d,:d] = g
    ret_list.append(new_g)

  return ret_list


In [0]:
max_tot = max(pos_max_len,neg_max_len)
pos_graphs_l = make_graphs_same_length(pos_graphs,max_tot)
neg_graphs_l = make_graphs_same_length(neg_graphs,max_tot)

In [0]:
with open('pos_graph_200.txt', 'w') as f:
    for item in pos_graphs_l:
        f.write("%s\n" % item)
with open('neg_graph_200.txt', 'w') as f:
    for item in neg_graphs_l:
        f.write("%s\n" % item)    

In [0]:
from google.colab import files
files.download('pos_graph_200.txt')

In [0]:
files.download('neg_graph_200.txt')